### Simulating Physics Based Model

In [1]:
import pybamm
import numpy as np

In [2]:
def my_current(t):
    current = pybamm.sin(2 * np.pi * t / 60)
    return current

In [3]:
options = {"thermal": "x-full"}
model = pybamm.lithium_ion.SPMe(options=options) 

In [4]:
parameter_values = model.default_parameter_values
parameter_values

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Current function [A]': 0.680616,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.13699999999999998,
 'Electrode width [m]': 0.207,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x00000146CE5FD280>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x00000146CE5FD3A0>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 19986.609595075,
 'Initial concentration in positive electrode [mol.m-3]': 30730.755438556498,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'I

In [5]:
# defining RMSE
import math

def RMSE(true_voltage, model_voltage):
    res = 0
    n = len(true_voltage)
    for i in range(n):
        res+= (true_voltage[i]-model_voltage[i])**2
    res = res/n
    return math.sqrt(res)

In [6]:
# defining array of C_rates
C_Rates = [0.1, 0.2, 0.4, 0.8, 1, 2, 4, 6, 8, 10]


NSOC = []
Temp = []
Voltage = []
OCV = []
C_Rate = []

SPMT_RMSE = []

n = len(C_Rates)


for i in range (n):
    sim = pybamm.Simulation(model, parameter_values=parameter_values, C_rate = C_Rates[i])
    sim.solve([0,3600])
    nsoc = sim.solution["Negative electrode SOC"].entries
    temp = sim.solution["Negative electrode temperature [K]"].entries[-1]
    voltage = sim.solution["Terminal voltage [V]"].entries
    ocv = sim.solution["Measured open circuit voltage [V]"].entries
    
    rmse = RMSE(ocv,voltage) 
    SPMT_RMSE.append(rmse)
    
    for j in range(len(nsoc)):
        C_Rate.append(C_Rates[i])
        NSOC.append(nsoc[j])
        Temp.append(temp[j])
        Voltage.append(voltage[j])
        OCV.append(ocv[j])
        
        

In [7]:
# creating a dataframe and storing the results

import pandas as pd

df = pd.DataFrame()
df['C_Rate'] = C_Rate
df['NSOC'] = NSOC
df['Temp'] = Temp
df['Voltage'] = Voltage
df['OCV'] = OCV

In [13]:
df.head()

,C_Rate,NSOC,Temp,Voltage,OCV
0,0.1,0.800000,298.150000,3.841727,3.851824
1,0.1,0.799397,298.153985,3.840125,3.850778
2,0.1,0.798793,298.158054,3.839417,3.850216
3,0.1,0.798190,298.162147,3.838888,3.849725
4,0.1,0.797586,298.166249,3.838417,3.849262


In [14]:
# storing result into spmt.csv file
df.to_csv(r"spmt_data.csv")

In [11]:
# Creating C_Rate VS RMSE csv
df2 = pd.DataFrame()
df2['C_Rate'] = C_Rates
df2['SPMT_RMSE'] = SPMT_RMSE
df2.head(10)

,C_Rate,SPMT_RMSE
0,0.1,0.010783
1,0.2,0.021097
2,0.4,0.038985
3,0.8,0.063912
4,1.0,0.073439
5,2.0,0.107727
6,4.0,0.150256
7,6.0,0.179473
8,8.0,0.206541
9,10.0,0.226304


In [12]:
# storing result into spmt_rmse.csv
df2.to_csv(r"spmt_rmse.csv")

### Simulating Hybrid Model

In [33]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"spmt_data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[3,4,5]], axis = 1)
y = df.drop(df.columns[[0,1,2,3,4]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

   Unnamed: 0  C_Rate      NSOC
0           0     0.1  0.800000
1           1     0.1  0.799397
2           2     0.1  0.798793
3           3     0.1  0.798190
4           4     0.1  0.797586
        OCV
0  3.851824
1  3.850778
2  3.850216
3  3.849725
4  3.849262


In [34]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(32, activation = 'relu', input_dim = 3),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 250)


Epoch 1/250
16/16 [==============================] - 1s 1ms/step - loss: 8864.8834 - mean_squared_error: 8864.8834
Epoch 2/250
16/16 [==============================] - 0s 1ms/step - loss: 721.3788 - mean_squared_error: 721.3788
Epoch 3/250
16/16 [==============================] - 0s 533us/step - loss: 97.8886 - mean_squared_error: 97.8886
Epoch 4/250
16/16 [==============================] - 0s 533us/step - loss: 39.9510 - mean_squared_error: 39.9510
Epoch 5/250
16/16 [==============================] - 0s 533us/step - loss: 7.8772 - mean_squared_error: 7.8772
Epoch 6/250
16/16 [==============================] - 0s 1ms/step - loss: 5.2978 - mean_squared_error: 5.2978
Epoch 7/250
16/16 [==============================] - 0s 1ms/step - loss: 4.1662 - mean_squared_error: 4.1662
Epoch 8/250
16/16 [==============================] - 0s 1ms/step - loss: 3.8590 - mean_squared_error: 3.8590
Epoch 9/250
16/16 [==============================] - 0s 1ms/step - loss: 3.9700 - mean_squared_error: 3.9700

16/16 [==============================] - 0s 3ms/step - loss: 0.0145 - mean_squared_error: 0.0145
Epoch 150/250
16/16 [==============================] - 0s 3ms/step - loss: 0.0192 - mean_squared_error: 0.0192
Epoch 151/250
16/16 [==============================] - 0s 3ms/step - loss: 0.0209 - mean_squared_error: 0.0209
Epoch 152/250
16/16 [==============================] - 0s 3ms/step - loss: 0.0183 - mean_squared_error: 0.0183
Epoch 153/250
16/16 [==============================] - 0s 2ms/step - loss: 0.0165 - mean_squared_error: 0.0165
Epoch 154/250
16/16 [==============================] - 0s 3ms/step - loss: 0.0238 - mean_squared_error: 0.0238
Epoch 155/250
16/16 [==============================] - 0s 2ms/step - loss: 0.0135 - mean_squared_error: 0.0135
Epoch 156/250
16/16 [==============================] - 0s 2ms/step - loss: 0.0252 - mean_squared_error: 0.0252
Epoch 157/250
16/16 [==============================] - 0s 3ms/step - loss: 0.0191 - mean_squared_error: 0.0191
Epoch 158/250
1

In [35]:
score = model.evaluate(x_test, y_test)
score

4/4 [==============================] - 0s 4ms/step - loss: 0.0100 - mean_squared_error: 0.0100


[0.010027347132563591, 0.010027347132563591]

In [36]:
y_predicted = model.predict(x)
df['Hybrid_Predicted_Voltage'] = y_predicted
df.head()

,Unnamed: 0,C_Rate,NSOC,Temp,Voltage,OCV,Hybrid_Predicted_Voltage
0,0,0.1,0.800000,298.150000,3.841727,3.851824,3.860100
1,1,0.1,0.799397,298.153985,3.840125,3.850778,3.782901
2,2,0.1,0.798793,298.158054,3.839417,3.850216,3.843900
3,3,0.1,0.798190,298.162147,3.838888,3.849725,3.869452
4,4,0.1,0.797586,298.166249,3.838417,3.849262,3.851907


In [37]:
# storing result into hybrid.csv file
df.to_csv(r"hybrid_data.csv")

In [38]:
Hybrid_RMSE = []

m = len(df['C_Rate'])

for i in range(n):
    c_rate = C_Rates[i]
    ocv = []
    hybrid = []
    for j in range(m):
        if df['C_Rate'][j] == c_rate:
            ocv.append(df['OCV'][j])
            hybrid.append(df['Hybrid_Predicted_Voltage'][j])
            
    rmse = RMSE(ocv , hybrid)
    Hybrid_RMSE.append(rmse)


In [39]:
data = pd.read_csv(r"spmt_rmse.csv")
df2 = pd.DataFrame(data)
df2['Hybrid_RMSE'] = Hybrid_RMSE
df2.head(10)

,Unnamed: 0,C_Rate,SPMT_RMSE,Hybrid_RMSE
0,0,0.1,0.010783,0.014332
1,1,0.2,0.021097,0.020271
2,2,0.4,0.038985,0.025802
3,3,0.8,0.063912,0.094791
4,4,1.0,0.073439,0.160272
5,5,2.0,0.107727,0.165108
6,6,4.0,0.150256,0.110694
7,7,6.0,0.179473,0.121411
8,8,8.0,0.206541,0.135617
9,9,10.0,0.226304,0.140963


In [40]:
def RER(RMSE_SPMT,RMSE_Hybrid):
    rer = ((RMSE_SPMT - RMSE_Hybrid)/RMSE_SPMT) * 100
    return rer

In [41]:
ReR = []

m = len(df2['SPMT_RMSE'])
for i in range(m):
    rer = RER(df2['SPMT_RMSE'][i],df2['Hybrid_RMSE'][i])
    ReR.append(rer)
    
df2['RER']=ReR 

In [42]:
# storing result into hybrid_rmse.csv
df2.to_csv(r"hybrid_rmse.csv")

In [43]:
df2.head(10)

,Unnamed: 0,C_Rate,SPMT_RMSE,Hybrid_RMSE,RER
0,0,0.1,0.010783,0.014332,-32.912043
1,1,0.2,0.021097,0.020271,3.914582
2,2,0.4,0.038985,0.025802,33.814043
3,3,0.8,0.063912,0.094791,-48.314656
4,4,1.0,0.073439,0.160272,-118.238872
5,5,2.0,0.107727,0.165108,-53.265368
6,6,4.0,0.150256,0.110694,26.329824
7,7,6.0,0.179473,0.121411,32.351405
8,8,8.0,0.206541,0.135617,34.338912
9,9,10.0,0.226304,0.140963,37.710943
